In [1]:
import sys
sys.path.append('/Users/paullin/Documents/numerai')

from utils.utils import *

In [2]:
import json
napi = NumerAPI()
DATA_VERSION = 'v5.0'
feature_metadata = json.load(open(f'../data/{DATA_VERSION}/features.json'))
feature_set = feature_metadata['feature_sets']['small']

import pandas as pd

train = pd.read_parquet(
    f'../data/{DATA_VERSION}/train.parquet',
    columns=['era', 'target'] + feature_set
)

X_train, y_train = train[feature_set], train['target'].to_frame()

In [3]:
X_train

,feature_antistrophic_striate_conscriptionist,feature_bicameral_showery_wallaba,feature_bridal_fingered_pensioner,feature_collectivist_flaxen_gueux,feature_concurring_fabled_adapter,feature_crosscut_whilom_ataxy,feature_departmental_inimitable_sentencer,feature_dialectal_homely_cambodia,feature_donnard_groutier_twinkle,feature_elusive_vapoury_accomplice,...,feature_tridactyl_immoral_snorting,feature_trimeter_soggy_greatest,feature_unanalyzable_excusable_whirlwind,feature_unbreakable_constraining_hegelianism,feature_unformed_bent_smatch,feature_unministerial_unextenuated_teleostean,feature_unmodish_zymogenic_rousing,feature_unsystematized_subcardinal_malaysia,feature_willful_sere_chronobiology,feature_zoological_peristomial_scute
id,,,,,,,,,,,,,,,,,,,,,
n0007b5abb0c3a25,2,2,2,2,2,0,1,2,2,1,...,4,1,1,3,0,2,2,3,3,2
n003bba8a98662e4,2,2,2,2,2,1,4,2,2,4,...,4,2,0,0,0,2,2,4,4,2
n003bee128c2fcfc,2,2,2,2,2,2,2,2,2,0,...,3,1,1,0,1,2,2,0,3,2
n0048ac83aff7194,2,2,2,2,2,1,4,2,2,3,...,1,3,4,1,2,2,2,2,0,2
n0055a2401ba6480,2,2,2,2,2,0,0,2,2,2,...,1,0,1,0,0,2,2,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffcc1dbdf2212e6,1,3,0,0,0,4,3,1,1,4,...,2,3,4,2,4,3,2,3,3,3
nffde3b371d67394,2,3,4,4,2,2,4,2,2,1,...,2,4,4,3,4,2,1,4,4,2
nfff1a1111b35e84,0,0,2,1,4,0,0,4,3,1,...,4,0,0,0,0,4,4,0,0,2


In [4]:
import json
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import GridSearchCV

model = lgb.LGBMRegressor(
  n_estimators=10,
  learning_rate=0.01,
  max_depth=5,
  num_leaves=2**5-1,
  colsample_bytree=0.1
)

model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008275 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 210
[LightGBM] [Info] Number of data points in the train set: 2746270, number of used features: 42
[LightGBM] [Info] Start training from score 0.500027


LGBMRegressor(colsample_bytree=0.1, learning_rate=0.01, max_depth=5,
              n_estimators=10)

In [5]:
validation = pd.read_parquet(
    f"../data/{DATA_VERSION}/validation.parquet",
    columns=["era", "data_type", "target"] + feature_set
)
validation = validation[validation["data_type"] == "validation"]
del validation["data_type"]

validation = validation[validation["era"].isin(validation["era"].unique()[::4])]

last_train_era = int(train["era"].unique()[-1])
eras_to_embargo = [str(era).zfill(4) for era in [last_train_era + i for i in range(4)]]
validation = validation[~validation["era"].isin(eras_to_embargo)]

validation["prediction"] = model.predict(validation[feature_set])
validation[["era", "prediction", "target"]]

,era,prediction,target
id,,,
n000c290e4364875,0579,0.499920,0.50
n002a15bc5575bbb,0579,0.500346,0.25
n00309caaa0f955e,0579,0.500440,0.75
n0039cbdcf835708,0579,0.500010,0.50
n004143458984f89,0579,0.499795,0.50
...,...,...,...
nffef5148b95d67a,1143,0.499894,0.50
nffef7c5973c9171,1143,0.500019,0.50
nfff634a1a813cd3,1143,0.499918,0.50


In [6]:
x = score(validation, DATA_VERSION)

In [12]:
x

{'corr': {'mean': np.float64(0.010259340333123838),
  'std': np.float64(0.016303625663754602),
  'sharpe': np.float64(0.6292674123358883),
  'max_drawdown': np.float64(0.035837216141946016)},
 'mmc': {'mean': np.float64(-0.00513963837884874),
  'std': np.float64(0.019235141428300895),
  'sharpe': np.float64(-0.26720044653722835),
  'max_drawdown': np.float64(0.005137354464327098)}}

In [17]:
def check_churn(scores):
    # Import stats

    for t, value in scores.items():
        for stat, v in value.items():
            # compare stat with old

    # if something wrong, retrain

In [18]:
check_churn(x)

0.010259340333123838
0.016303625663754602
0.6292674123358883
0.035837216141946016
-0.00513963837884874
0.019235141428300895
-0.26720044653722835
0.005137354464327098
